# Tarefa 03

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [13]:
##01 - selecione os PVLs realizados por municípios no estado do Rio de Janeiro.

import requests
import pandas as pd

# URL da API
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

# Fazer a requisição para a API
response = requests.get(url)
dados = response.json()

# Extrair dados relevantes (ajuste conforme a estrutura)
dados_relevantes = dados['items']  

# Criar o DataFrame
dados_pvl = pd.DataFrame(dados_relevantes)

# Filtrar PVLs de municípios do RJ
filtro_rj_municipios = (dados_pvl['uf'] == 'RJ') & (dados_pvl['tipo_interessado'] == 'Município')
dados_filtrados = dados_pvl[filtro_rj_municipios]

# Exibir os resultados
print(dados_filtrados.head())

# Salvar em CSV 
dados_filtrados.to_csv("pvl_municipios_rj.csv", index=False)




     id_pleito tipo_interessado     interessado  cod_ibge  uf  \
389      34100        Município         Resende   3304201  RJ   
504      33609        Município       Vassouras   3306206  RJ   
660      28822        Município         Resende   3304201  RJ   
793      27100        Município  Rio de Janeiro   3304557  RJ   
902      60354        Município          Paraty   3303807  RJ   

                  num_pvl                          status  \
389  PVL02.003091/2019-33               Deferido (PVL-IF)   
504  PVL02.002481/2019-96                        Deferido   
660  PVL02.001389/2018-28                        Deferido   
793  PVL02.002429/2017-78  Arquivado por decurso de prazo   
902  PVL02.006783/2022-39               Deferido (PVL-IF)   

             num_processo        data_protocolo                tipo_operacao  \
389                  None                  None  Operação contratual interna   
504  17944.102288/2019-17  2019-07-29T14:06:25Z  Operação contratual interna   
66

##02- Quais são os três status das solicitações mais frequentes na base? Quais são suas frequências?


In [37]:
#exibindo colunas
print(dados_pvl.columns)


Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')


In [39]:
# Contar a frequência de cada status
status_frequencia = dados_pvl['status'].value_counts()
top3_status = status_frequencia.head(3)

#exibindo resultado
print(top3_status)


status
Deferido             2497
Deferido (PVL-IF)    1326
Arquivado             348
Name: count, dtype: int64


##03- Construa uma nova variável que contenha o ano do status. Observe que data_status vem como tipo object no DataFrame. Dica: você pode usar o método .str para transformar o tipo da variável em string, em seguida um método como slice() ou split().

In [55]:
# Certifique-se de que 'data_status' é tratada como string
dados_pvl['data_status'] = dados_pvl['data_status'].astype(str)

# Extrair o ano da coluna 'data_status' (supondo o formato 'dd/mm/aaaa')
dados_pvl['ano_status'] = dados_pvl['data_status'].str.slice(start=6, stop=10)

# Exibir as primeiras linhas para confirmar o resultado
print(dados_pvl[['data_status', 'ano_status']].head())


  data_status ano_status
0  27/07/2021       2021
1  31/03/2020       2020
2  07/07/2020       2020
3  24/06/2022       2022
4  30/12/2019       2019


##04- Indique a frequência de cada ano do campo construído no item (3).
Selection deleted


In [66]:
#calculando frequencia de ano_status
frequencia_anos = dados_pvl['ano_status'].value_counts()

#exibindo resultado
print(frequencia_anos)

ano_status
2019    521
2018    408
2008    389
2007    261
2021    259
2017    250
2022    248
2023    246
2012    228
2014    220
2010    218
2020    218
2009    186
2011    178
2013    177
2024    173
2006    172
2016    149
2015    136
2004    134
2002    105
2003     82
2005     42
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [3]:
import requests
import pandas as pd

# URL da API
url = ' http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'


# Fazer a requisição para a API
response = requests.get(url)
dados = response.json()

# Extrair dados relevantes (ajuste conforme a estrutura)
dados_relevantes = dados['items']  

# Criar o DataFrame
dados_pvl = pd.DataFrame(dados_relevantes)

# Filtrar PVLs de municípios do RJ
filtro_rj_municipios = (dados_pvl['uf'] == 'RJ') & (dados_pvl['tipo_interessado'] == 'Município')
dados_filtrados = dados_pvl[filtro_rj_municipios]

# Exibir os resultados
print(dados_filtrados.head())

# Salvar em CSV 
dados_filtrados.to_csv("pvl_municipios_rj.csv", index=False)



Empty DataFrame
Columns: [id_pleito, tipo_interessado, interessado, cod_ibge, uf, num_pvl, status, num_processo, data_protocolo, tipo_operacao, finalidade, tipo_credor, credor, moeda, valor, pvl_assoc_divida, pvl_contradado_credor, data_status]
Index: []


In [51]:
 #01 - Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado 
#('Estado'ou Município), e que devolve os dados da consulta no formato DataFrame.

import requests
import pandas as pd

def consulta_pvl(uf, tipo_interessado):
    """
    Consulta os dados da API de PVL (Proposta de Valores Líquidos) do Tesouro Nacional
    e retorna os dados no formato DataFrame.

    Parâmetros:
    - uf (str): Unidade Federativa (exemplo: 'SP', 'PB')
    - tipo_interessado (str): Tipo de interessado ('Estado' ou 'Município')

    Retorno:
    - DataFrame com os dados filtrados pela UF e tipo de interessado.
    """
    # URL da API
    url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Realiza a requisição para a API
    response = requests.get(url)
    
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Converte a resposta JSON em DataFrame
        dados = response.json()
        df = pd.DataFrame(dados['items'])
        
        # Filtra os dados pela UF e tipo de interessado
        df_filtrado = df[(df['uf'] == uf) & (df['tipo_interessado'] == tipo_interessado)]
        return df_filtrado
    else:
        print(f"Erro na requisição: {response.status_code}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro


In [55]:
# Consulta para a UF 'MG' e tipo de interessado 'Município'
df_resultado = consulta_pvl('MG', 'Município')

# Exibir os primeiros registros
print(df_resultado.head())


    id_pleito tipo_interessado              interessado  cod_ibge  uf  \
1        8238        Município  Santo Antônio do Amparo   3159902  MG   
3       27599        Município                  Raposos   3153905  MG   
4       26604        Município            Sapucaí-Mirim   3165404  MG   
17      26752        Município                Rio Manso   3155306  MG   
23      27674        Município                Juramento   3136801  MG   

                 num_pvl             status          num_processo  \
1                   None           Deferido  17944.000739/2009-19   
3   PVL02.000252/2018-56  Deferido (PVL-IF)                  None   
4   PVL02.002304/2017-48  Deferido (PVL-IF)                  None   
17  PVL02.002254/2017-07           Deferido  17944.102810/2017-07   
23  PVL02.000327/2018-07  Deferido (PVL-IF)                  None   

          data_protocolo                tipo_operacao  \
1   2009-06-29T00:00:00Z  Operação contratual interna   
3                   None  Operaç

In [71]:
#02 - Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?
import pandas as pd
import requests

def consulta_pvl_contagem(uf, status):
    """
    Consulta e conta as solicitações na API de PVL do Tesouro Nacional 
    para uma UF e status específicos.

    Parâmetros:
    - uf (str): Unidade Federativa (exemplo: 'MG')
    - status (str): Status das solicitações (exemplo: 'Arquivado por decurso de prazo')

    Retorno:
    - Inteiro com a contagem de solicitações correspondentes.
    """
    # URL da API
    url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Realiza a requisição para a API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Processa os dados da API
        dados = response.json()
        df = pd.DataFrame(dados['items'])
        
        # Filtra os dados por UF e Status
        df_filtrado = df[(df['uf'] == uf) & (df['status'] == status)]
        
        # Retorna a contagem
        return len(df_filtrado)
    else:
        print(f"Erro na requisição: {response.status_code}")
        return 0


In [73]:
# Consultar solicitações para Minas Gerais com status 'Arquivado por decurso de prazo'
uf = 'MG'
status = 'Arquivado por decurso de prazo'
contagem = consulta_pvl_contagem(uf, status)

print(f"Quantidade de solicitações: {contagem}")


Quantidade de solicitações: 63


In [75]:
#03 - Qual é o município da Bahia com mais solicitações deferidas?

def municipio_com_mais_deferidas(uf, status):
    """
    Consulta os dados da API de PVL do Tesouro Nacional e encontra o município
    com o maior número de solicitações com o status especificado.

    Parâmetros:
    - uf (str): Unidade Federativa (exemplo: 'BA')
    - status (str): Status das solicitações (exemplo: 'Deferido')

    Retorno:
    - Nome do município com mais solicitações deferidas e a quantidade.
    """
    # URL da API
    url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Realiza a requisição para a API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Processa os dados da API
        dados = response.json()
        df = pd.DataFrame(dados['items'])
        
        # Filtra os dados por UF e Status
        df_filtrado = df[(df['uf'] == uf) & (df['status'] == status)]
        
        # Agrupa por município e conta as solicitações
        municipio_mais_solicitacoes = df_filtrado['interessado'].value_counts().idxmax()
        quantidade = df_filtrado['interessado'].value_counts().max()
        
        return municipio_mais_solicitacoes, quantidade
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None, 0


In [77]:
# Consultar o município da Bahia com mais solicitações deferidas
uf = 'BA'
status = 'Deferido'
municipio, quantidade = municipio_com_mais_deferidas(uf, status)

print(f"Município com mais solicitações deferidas: {municipio} ({quantidade} solicitações)")


Município com mais solicitações deferidas: Bahia (10 solicitações)


In [79]:
#04 - Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

def salvar_solicitacoes_bahia_estado_em_csv():
    """
    Consulta os dados de solicitações para a Bahia com interessado 'Estado'
    e salva os resultados em um arquivo CSV.

    Retorno:
    - Nome do arquivo salvo.
    """
    import os
    
    # URL da API
    url = "http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Realiza a requisição para a API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Processa os dados da API
        dados = response.json()
        df = pd.DataFrame(dados['items'])
        
        # Filtra os dados por UF = 'BA' e tipo_interessado = 'Estado'
        df_filtrado = df[(df['uf'] == 'BA') & (df['tipo_interessado'] == 'Estado')]
        
        # Nome do arquivo para salvar
        arquivo_csv = "solicitacoes_bahia_estado.csv"
        
        # Salva os dados em CSV
        df_filtrado.to_csv(arquivo_csv, index=False, encoding='utf-8')
        
        print(f"Arquivo salvo: {os.path.abspath(arquivo_csv)}")
        return arquivo_csv
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Executar a função
salvar_solicitacoes_bahia_estado_em_csv()


Arquivo salvo: C:\Users\fabri\EBAC\atividade04\solicitacoes_bahia_estado.csv


'solicitacoes_bahia_estado.csv'